# Classificação de Textos por Estilo Literário

Este projeto utiliza algoritmos de aprendizado supervisionado para classificar textos 
de acordo com seus estilos literários: Poesia, Prosa e Jornalismo.

## Etapas:
1. Carregar os dados.
2. Pré-processar os textos.
3. Vetorizar os textos (Bag of Words).
4. Treinar e avaliar os seguintes modelos:
   - Decision Tree
   - K-Nearest Neighbors
   - Naive Bayes
   - Logistic Regression
   - Multi-layer Perceptron
5. Comparar os resultados.


# Parte 1

In [6]:
%pip install PyPDF2 scikit-learn nltk pandas matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importação das Bibliotecas

In [14]:
# ===============================
# Célula 1: Importação das Bibliotecas
# ===============================
import os
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Baixar pacotes necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Função para Extração de Texto de PDFs

In [ ]:
def pdf_para_txt(caminho_pdf):
    """
    Lê um arquivo PDF e retorna o texto extraído.
    """
    with open(caminho_pdf, 'rb') as f:
        leitor = PyPDF2.PdfReader(f)
        texto = ""
        for pagina in range(len(leitor.pages)):
            texto += leitor.pages[pagina].extract_text()
    return texto


Função para Limpeza e Remoção de Stopwords

In [ ]:
def limpar_texto(texto):
    """
    Limpa o texto, remove stopwords e caracteres especiais.
    """
    stop_words = set(stopwords.words('portuguese'))
    palavras = word_tokenize(texto.lower())
    palavras_limpa = [palavra for palavra in palavras if palavra.isalnum() and palavra not in stop_words]
    return " ".join(palavras_limpa)


Configuração dos Diretórios

In [17]:
# ===============================
# Célula 4: Configuração dos Diretórios
# ===============================
# Diretórios com os PDFs (ajuste os caminhos conforme necessário)
diretorios = {
    'poesia': 'pdfs/poesia/',
    'prosa': 'pdfs/prosa/',
    'jornalismo': 'pdfs/jornalismo/'
}

# Pasta base para salvar os textos extraídos
output_dir = 'textos_extraidos'

# Criar diretório principal e subpastas
os.makedirs(output_dir, exist_ok=True)
for classe in diretorios.keys():
    os.makedirs(os.path.join(output_dir, classe), exist_ok=True)


Extração de Texto e Salvamento em Arquivos TXT

In [18]:
# ===============================
# Célula 5: Extração de Texto e Salvamento em Arquivos TXT
# ===============================
textos = []
classes = []

for classe, caminho in diretorios.items():
    for arquivo in os.listdir(caminho):
        if arquivo.endswith('.pdf'):
            # Extrair e limpar texto do PDF
            texto = pdf_para_txt(os.path.join(caminho, arquivo))
            texto_limpo = limpar_texto(texto)
            
            # Caminho para salvar o arquivo de texto na subpasta correspondente
            subpasta = os.path.join(output_dir, classe)
            output_path = os.path.join(subpasta, arquivo.replace('.pdf', '.txt'))
            
            # Salvar texto limpo no arquivo .txt
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(texto_limpo)
            
            # Armazenar texto e classe para uso posterior
            textos.append(texto_limpo)
            classes.append(classe)

print(f"Textos extraídos e organizados em subpastas de '{output_dir}'.")


Textos extraídos e organizados em subpastas de 'textos_extraidos'.


Criação da Matriz Bag of Words (BoW)

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textos)

print("Matriz BoW criada com sucesso!")
print("Dimensões da matriz:", X.shape)


Matriz BoW criada com sucesso!
Dimensões da matriz: (291, 214536)


Divisão dos Dados em Treino e Teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size=0.3, random_state=42)
print("Dados divididos em treino e teste.")


Dados divididos em treino e teste.


# Parte 2

#### Coleta de Medidas de Desempenho com validação cruzada

In [15]:
from sklearn.model_selection import cross_val_score

def treinar_modelo_com_crossval(modelo, nome_modelo):
    """
    Treina e avalia o modelo usando cross-validation com 10 folds.
    Retorna as médias das métricas de avaliação.
    """
    # Calculando acurácia
    accuracies = cross_val_score(modelo, X_train, y_train, cv=10, scoring='accuracy')
    # Calculando F1-Score
    f1_scores = cross_val_score(modelo, X_train, y_train, cv=10, scoring='f1_weighted')

    resultado = {
        'nome_modelo': nome_modelo,
        'acuracia_media': np.mean(accuracies),
        'acuracia_std': np.std(accuracies),
        'f1_media': np.mean(f1_scores),
        'f1_std': np.std(f1_scores)
    }

    print(f"\n{nome_modelo}:")
    print(f"Acurácia média: {resultado['acuracia_media']:.4f} (+/- {resultado['acuracia_std']:.4f})")
    print(f"F1-Score médio: {resultado['f1_media']:.4f} (+/- {resultado['f1_std']:.4f})")
    
    return resultado


KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print("Treinando modelo: KNN")
modelo_knn = KNeighborsClassifier()
treinar_modelo_com_crossval(modelo_knn, "KNN")


Treinando modelo: KNN

KNN:
Acurácia média: 0.5652 (+/- 0.0931)
F1-Score médio: 0.5333 (+/- 0.1269)


{'nome_modelo': 'KNN',
 'acuracia_media': 0.5652380952380952,
 'acuracia_std': 0.09311441770521092,
 'f1_media': 0.5332767695267695,
 'f1_std': 0.12694557197197834}

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

print("Treinando modelo: Decision Tree")
modelo_dt = DecisionTreeClassifier()
treinar_modelo_com_crossval(modelo_dt, "Decision Tree")


Treinando modelo: Decision Tree

Decision Tree:
Acurácia média: 0.8029 (+/- 0.0630)
F1-Score médio: 0.7832 (+/- 0.1058)


{'nome_modelo': 'Decision Tree',
 'acuracia_media': 0.8028571428571427,
 'acuracia_std': 0.06302826634357513,
 'f1_media': 0.7832106550856551,
 'f1_std': 0.10582272137149301}

Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

print("Treinando modelo: Naive Bayes")
modelo_nb = MultinomialNB()
treinar_modelo_com_crossval(modelo_nb, "Naive Bayes")


Treinando modelo: Naive Bayes

Naive Bayes:
Acurácia média: 0.7031 (+/- 0.0976)
F1-Score médio: 0.6990 (+/- 0.1049)


{'nome_modelo': 'Naive Bayes',
 'acuracia_media': 0.7030952380952381,
 'acuracia_std': 0.09758448865626061,
 'f1_media': 0.6990087302331884,
 'f1_std': 0.10489116240046446}

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

print("Treinando modelo: Logistic Regression")
modelo_lr = LogisticRegression(max_iter=1000)
treinar_modelo_com_crossval(modelo_lr, "Logistic Regression")


Treinando modelo: Logistic Regression

Logistic Regression:
Acurácia média: 0.8669 (+/- 0.0550)
F1-Score médio: 0.8638 (+/- 0.0574)


{'nome_modelo': 'Logistic Regression',
 'acuracia_media': 0.866904761904762,
 'acuracia_std': 0.05499381536037624,
 'f1_media': 0.863755689571866,
 'f1_std': 0.057359480635165785}

MLP Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

print("Treinando modelo: MLP Classifier")
modelo_mlp = MLPClassifier(max_iter=500, early_stopping=True, n_iter_no_change=10, verbose=True)
treinar_modelo_com_crossval(modelo_mlp, "MLP Classifier")


Treinando modelo: MLP Classifier
Iteration 1, loss = 1.10145265
Validation score: 0.210526
Iteration 2, loss = 1.07186477
Validation score: 0.210526
Iteration 3, loss = 1.03629069
Validation score: 0.210526
Iteration 4, loss = 0.99727448
Validation score: 0.263158
Iteration 5, loss = 0.95692242
Validation score: 0.368421
Iteration 6, loss = 0.91575201
Validation score: 0.473684
Iteration 7, loss = 0.87467147
Validation score: 0.578947
Iteration 8, loss = 0.83431863
Validation score: 0.631579
Iteration 9, loss = 0.79452887
Validation score: 0.631579
Iteration 10, loss = 0.75527934
Validation score: 0.684211
Iteration 11, loss = 0.71675095
Validation score: 0.736842
Iteration 12, loss = 0.67904247
Validation score: 0.789474
Iteration 13, loss = 0.64248276
Validation score: 0.789474
Iteration 14, loss = 0.60723361
Validation score: 0.789474
Iteration 15, loss = 0.57335242
Validation score: 0.736842
Iteration 16, loss = 0.54090724
Validation score: 0.736842
Iteration 17, loss = 0.50996255


{'nome_modelo': 'MLP Classifier',
 'acuracia_media': 0.8416666666666666,
 'acuracia_std': 0.07030263378750697,
 'f1_media': 0.8144248077630429,
 'f1_std': 0.07380437077034988}

# Parte 3

#### Hiperparâmetros e Otimização

KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Parâmetros para otimizar no KNeighborsClassifier
parametros_knn = {
    'n_neighbors': [3, 5, 7, 9],  
    'weights': ['uniform', 'distance'],  
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
}

# GridSearchCV para KNeighborsClassifier
grid_search_knn = GridSearchCV(KNeighborsClassifier(), parametros_knn, 
                               cv=10, n_jobs=-1, verbose=2)
grid_search_knn.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados para KNN
print(f"Melhores parâmetros para o KNN: {grid_search_knn.best_params_}")

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Melhores parâmetros para o KNN: {'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'uniform'}


Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Parâmetros para otimizar na DecisionTree
parametros_dt = {
    'max_depth': [10, 20, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4]  
}

# GridSearchCV para DecisionTree
grid_search_dt = GridSearchCV(DecisionTreeClassifier(), parametros_dt, 
                            cv=10, n_jobs=-1, verbose=2)

# Fit do GridSearch no conjunto de treino
grid_search_dt.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados para DecisionTree
print(f"Melhores parâmetros para a Decision Tree: {grid_search_dt.best_params_}")

Fitting 10 folds for each of 27 candidates, totalling 270 fits
Melhores parâmetros para a Decision Tree: {'max_depth': 30, 'min_samples_leaf': 4, 'min_samples_split': 2}


Naive Bayes

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Parâmetros para otimizar no Naive Bayes
parametros_nb = {
    'alpha': [0.1, 0.5, 1.0]  
}

# GridSearchCV para Naive Bayes
grid_search_nb = GridSearchCV(MultinomialNB(), parametros_nb, 
                            cv=10, n_jobs=-1, verbose=2)

# Fit do GridSearch no conjunto de treino
grid_search_nb.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados para Naive Bayes
print(f"Melhores parâmetros para o Naive Bayes: {grid_search_nb.best_params_}")

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Melhores parâmetros para o Naive Bayes: {'alpha': 0.1}


Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Definindo o espaço de hiperparâmetros para o GridSearch
parametros_lr = {
    'C': [0.01, 0.1, 1, 10, 100],  
    'solver': ['liblinear', 'lbfgs', 'newton-cg'],  
    'max_iter': [100, 200, 300]  
}

# Aplicando GridSearchCV para otimizar os hiperparâmetros
grid_search_lr = GridSearchCV(LogisticRegression(), parametros_lr, 
                                cv=10, n_jobs=-1, verbose=2)

# Fit do GridSearch no conjunto de treino
grid_search_lr.fit(X_train, y_train)

# Mostrando os melhores hiperparâmetros encontrados
print("Melhores Hiperparâmetros para o Logistic Regression: ", grid_search_lr.best_params_)

Fitting 10 folds for each of 45 candidates, totalling 450 fits
Melhores Hiperparâmetros:  {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}


O MLPClassifier apresentou erros durante a execução de teste de Hiperparâmetros, meu PC travou completamente

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# Definindo a grade de parâmetros para o MLP
parametros_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],  
    'activation': ['relu', 'tanh'],  
    'alpha': [0.0001, 0.001, 0.01],  
    'learning_rate': ['constant', 'adaptive'],  
    'max_iter': [200, 300]  
}

# GridSearchCV para MLP
grid_search_mlp = GridSearchCV(MLPClassifier(), parametros_mlp, 
                            cv=10, n_jobs=-1, verbose=2)

# Fit do GridSearch no conjunto de treino
grid_search_mlp.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados
print(f"Melhores parâmetros para o MLP: {grid_search_mlp.best_params_}")

## Treino com Hiperparâmetros ajustados

KNN

In [ ]:
# Criando o modelo KNN com os melhores parâmetros encontrados
modelo_knn_otimizado = KNeighborsClassifier(
    algorithm='auto',       
    n_neighbors=3,
    weights='uniform'
)

# Treinando o modelo otimizado
modelo_knn_otimizado.fit(X_train, y_train)
y_pred_knn = modelo_knn_otimizado.predict(X_test)

# Calculando as métricas
print("Acurácia no conjunto de teste (KNN Otimizado):", accuracy_score(y_test, y_pred_knn))
print("F1-Score no conjunto de teste (KNN Otimizado):", f1_score(y_test, y_pred_knn, average='weighted'))


Acurácia no conjunto de teste (KNN Otimizado): 0.6818181818181818
F1-Score no conjunto de teste (KNN Otimizado): 0.686524500907441


Decision Tree

In [31]:
# Definindo o modelo com o melhor hiperparâmetro
modelo_dt_otimizado = DecisionTreeClassifier(
    max_depth=30,
    min_samples_leaf=4,
    min_samples_split=2
)

# Treinando o modelo no conjunto de treinamento
modelo_dt_otimizado.fit(X_train, y_train)
y_pred_dt = modelo_dt_otimizado.predict(X_test)

# Calculando as métricas
print("Acurácia no conjunto de teste (Decision Tree Otimizado):", accuracy_score(y_test, y_pred_dt))
print("F1-Score no conjunto de teste (Decision Tree Otimizado):", f1_score(y_test, y_pred_dt, average='weighted'))


Acurácia no conjunto de teste (Decision Tree Otimizado): 0.8068181818181818
F1-Score no conjunto de teste (Decision Tree Otimizado): 0.8053200468294808


Naive Bayes

In [ ]:
# Definindo o modelo com o melhor hiperparâmetro
modelo_nb_otimizado = MultinomialNB(alpha=0.1) 

# Treinando o modelo no conjunto de treinamento
modelo_nb_otimizado.fit(X_train, y_train)
y_pred_nb = modelo_nb_otimizado.predict(X_test)


print("Acurácia no conjunto de teste (Naive Bayes Otimizado):", accuracy_score(y_test, y_pred_nb))
print("F1-Score no conjunto de teste (Naive Bayes Otimizado):", f1_score(y_test, y_pred_nb, average='weighted'))

Avaliação no conjunto de teste (Naive Bayes Otimizado):
Acurácia: 0.9432
F1-Score: 0.9429


Logistic Regression

In [28]:
# Definindo o modelo com o melhor hiperparâmtro
modelo_lr_otimizado = LogisticRegression(C=10, 
                                         max_iter=100, 
                                         solver='liblinear')

# Treinando o modelo no conjunto de treinamento
modelo_lr_otimizado.fit(X_train, y_train)
y_pred_lr = modelo_lr_otimizado.predict(X_test)

print("Acurácia no conjunto de teste (Logistic Regression Otimizado):", accuracy_score(y_test, y_pred_lr))
print("F1-Score no conjunto de teste (Logistic Regression Otimizado):", f1_score(y_test, y_pred_lr, average='weighted'))


Acurácia no conjunto de teste (Logistic Regression Otimizado): 0.9431818181818182
F1-Score no conjunto de teste (Logistic Regression Otimizado): 0.9425037925037923


MLP

Na minha maquina não roda, trava tudo

In [ ]:
# Definindo o modelo com o melhor hiperparâmtro
modelo_mlp_otimizado = MLPClassifier(hidden_layer_sizes=(50,), 
                                     activation='relu',  
                                     solver='adam', 
                                     alpha=0.0001, 
                                     max_iter=200, 
                                     random_state=42)

# Treinando o modelo no conjunto de treinamento
grid_search_mlp.fit(X_train, y_train)
y_pred_mlp = modelo_mlp_otimizado.predict(X_test)

# Avaliando o desempenho do modelo otimizado
print("Acurácia no conjunto de teste (MLP Otimizado):", accuracy_score(y_test, y_pred_mlp))
print("F1-Score no conjunto de teste (MLP Otimizado):", f1_score(y_test, y_pred_mlp, average='weighted'))